# **Model Validation**

In [1]:
import openai
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key: ········


In [2]:
if 'OPENAI_API_KEY' in os.environ:
    openai_api_key = os.environ['OPENAI_API_KEY']
    print("OpenAI API Key: is set")
else:
    print("OPENAI_API_KEY environment variable is not set.")

OpenAI API Key: is set


In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
import pandas as pd
import numpy as np
import ast  # Import the ast module for literal evaluation
import seaborn as sns

pd.set_option('display.max_colwidth', 100)
# %pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

Available CPUs: 8
INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


-----
## Retrieval Dataframe

In [5]:
df_messages=pd.read_csv('gs://user-scripts-msca310019-capstone-49b3/data/20231019_Emails_with_Sender_Reply.csv', parse_dates=['time'])
df_messages.head(5)

,Unnamed: 0,Email_ID,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message,Sender_Receiver_Emails
0,0,0,1999-06-09 04:18:00-07:00,RE: test,2,5552,[40034],How about this Friday ? Julie has not left yet. She was 2 days away from leaving to start school...,1999-06-09 08:06:00-07:00,40034,[5552],when? how are you and your family? is julie gone?,NaN
1,1,1,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-06-10 03:54:00-07:00,5552,[40034],Today is bad. Tommorrow I will call you.,"['Do you have lunch plans today?', ""Really? I'd feel like a mooch. Lets have lunch next week. An..."
2,2,2,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-11-23 01:38:00-08:00,5552,[40034],Do you have lunch plans today?,"['Today is bad. Tommorrow I will call you.', ""Really? I'd feel like a mooch. Lets have lunch nex..."
3,3,3,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-11-23 03:13:00-08:00,5552,[40034],Really? I'd feel like a mooch. Lets have lunch next week. Any day but Monday or Thurs.,"['Today is bad. Tommorrow I will call you.', 'Do you have lunch plans today?', ""Tues.is good. I'..."
4,4,4,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-11-23 03:58:00-08:00,5552,[40034],Tues.is good. I'll call you.,"['Today is bad. Tommorrow I will call you.', 'Do you have lunch plans today?', ""Really? I'd feel..."


### De-Duplication for Retrieval ##

In [13]:
df_messages_deduped=df_messages.drop_duplicates(subset='sender_replier_thread', keep='last')
# df_messages_deduped.to_csv('gs://user-scripts-msca310019-capstone-49b3/data/20231026_Emails_Deduped.csv', index=False)







In [28]:
df_messages_deduped.head()

,Unnamed: 0,Email_ID,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message,Sender_Receiver_Emails,Sender_Receiver_Emails_list,sender_replier_thread
0,0,0,1999-06-09 04:18:00-07:00,RE: test,2,5552,[40034],"How about this Friday ? Julie has not left yet. She was 2 days away from leaving to start school on June 7th when she, quite by accident, looked at the school schedule for the first time in several weeks. Now mind you she had lined up a babysitter ,lined up a place to stay, arranged to get a key to start the 7th. The schedule says the school starts the 14th. Pretty funny. How many years will it take her to live this down?. Enron Capital & Trade Resources Corp.",1999-06-09 08:06:00-07:00,40034,[5552],when? how are you and your family? is julie gone?,NaN,[],5552-40034
4,4,4,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-11-23 03:58:00-08:00,5552,[40034],Tues.is good. I'll call you.,"['Today is bad. Tommorrow I will call you.', 'Do you have lunch plans today?', ""Really? I'd feel like a mooch. Lets have lunch next week. Any day but Monday or Thurs.""]","[Today is bad. Tommorrow I will call you., Do you have lunch plans today?, Really? I'd feel like a mooch. Lets have lunch next week. Any day but Monday or Thurs.]",40034-5552
10,10,13,2001-11-08 14:18:15-08:00,test,2,39298,[402],test Dave,2001-11-09 09:03:31-08:00,402,[39298],"Dave, Take a look at this before you pass it on. I haven't looked at it in a while. Let me know if you get it and how it looks. Thanks, Kevin",NaN,[],39298-402
11,11,14,2001-11-09 09:03:31-08:00,RE: test,2,402,[39298],"Dave, Take a look at this before you pass it on. I haven't looked at it in a while. Let me know if you get it and how it looks. Thanks, Kevin",2001-11-09 12:06:40-08:00,39298,[402],looks good. Metz says it looks fine too. You'll probably get more questions about Pub Ventures than trading. A lady named Susan Mills should be calling you either tonight or this weekend at home. She'll set up your interview. Everything looks good from this end. Dave,NaN,[],402-39298
12,12,15,2001-11-13 09:08:48-08:00,test,2,39322,[402],test Do You Yahoo!? Find the one for you at Yahoo! Personals http://personals.yahoo.com,2001-11-15 09:27:59-08:00,402,[39322],"sorry, but i didn't get your message until late. i'll call you at home tonight.",NaN,[],39322-402


-----
## Ranking Framework

In [14]:
# Importing Packages
import chromadb
from chromadb.utils import embedding_functions

import langchain
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [16]:
# Setting Up Client
chroma_client = chromadb.PersistentClient(path="vectorstores")
chroma_client.heartbeat()

1698346341606559065

In [17]:
# Defining the Embedding Function
openai_ef = embedding_functions.OpenAIEmbeddingFunction(api_key = openai_api_key,
                                                        model_name="text-embedding-ada-002")
openai_ef

----
## Langchain MVP v2

In [18]:
# Email Environment
sender_id='171'
replier_id='414'
sender_email='Where is my gift?'
email_dataset=df_messages_deduped
vector_db_client=chroma_client

In [22]:
async def get_email_response_personalized(sender_id=sender_id,
                                    replier_id=replier_id,
                                    sender_email=sender_email,
                                    email_dataset=df_messages_deduped,
                                    api_key=openai_api_key,
                                    vector_db_client=chroma_client,
                                    embedding_function=OpenAIEmbeddings()):
    
    # First getting retrieved emails to understand conversation --------
    sender_replier_id='-'.join([sender_id, replier_id])
    previous_emails=(email_dataset[email_dataset.sender_replier_thread==sender_replier_id]['Sender_Receiver_Emails_list']).to_list()[0][-10:]
    previous_emails
    # Second, getting ranked responses as per context ------------------
        
        # Building the Langchain vectorstore using chroma collections
    user_vector_store = Chroma(
        client=vector_db_client, 
        collection_name='user'+str(replier_id),
        embedding_function=OpenAIEmbeddings())
        # Getting ranked responses using MMR
    found_rel_emails = await user_vector_store.amax_marginal_relevance_search(sender_email, k=10, fetch_k=10)
    list_rel_emails=[]
    for i, doc in enumerate(found_rel_emails):
        list_rel_emails.append(doc.page_content)
    list_rel_emails
    
    print(list_rel_emails)
        
    # Setting up LangChain
    llm=ChatOpenAI(temperature=0.1, model=llm_model, openai_api_key=openai_api_key)
    template_string="""Create a response to the following email {sender_email}, deriving context from these relevant emails {relevant_emails} ,using a conversation style similar to following past emails between these 2 users {prev_emails}"""
    prompt_template = ChatPromptTemplate.from_template(template=template_string)    
    llm_chain=LLMChain(llm=llm, prompt=prompt_template)
    print(llm_chain.run(sender_email=sender_email, prev_emails=previous_emails, relevant_emails=list_rel_emails))
    

In [23]:
results = await get_email_response_personalized(sender_id=sender_id,
                                    replier_id=replier_id,
                                    sender_email=sender_email,
                                    email_dataset=df_messages_deduped,
                                    api_key=openai_api_key,
                                    vector_db_client=chroma_client,
                                    embedding_function=OpenAIEmbeddings())
print(results)

['by the way where is my present ', 'i have a present ', 'cmon thats not true, what am i going to do withyour present ', 'when do I get it ', 'did you get good stuff for christmas ', 'no nothing, where are you ', 'what did you get ', 'how did you lose it ', 'no but I still have your stuff do you want to come and get it ', 'where are you ']
Hey there,

I'm sorry for the delay in getting your gift to you. I've been really busy with work and haven't had a chance to drop it off yet. But don't worry, I still have it and I'll make sure to get it to you as soon as possible.

To answer your other questions, I did get some good stuff for Christmas. How about you? Did you get anything good?

And as for the present I have for you, I didn't lose it. I just haven't had a chance to give it to you yet. I promise I'll get it to you soon.

Let me know if you want to come and pick up your other stuff that I have. I'm still in the same place as before. 

Take care!
None


## Code Productionizing

In [24]:
import pandas as pd
import ast
import chromadb
from chromadb.utils import embedding_functions
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import asyncio

# Setting up the chroma client
chroma_client = chromadb.PersistentClient(path="vectorstores")
openai_api_key = openai_api_key

async def get_email_response_personalized(sender_id, replier_id, sender_email, email_dataset, api_key, vector_db_client):    
    # First getting retrieved emails to understand conversation --------
    sender_replier_id='-'.join([sender_id, replier_id])
    previous_emails=(email_dataset[email_dataset.sender_replier_thread==sender_replier_id]['Sender_Receiver_Emails_list']).to_list()[0][-10:]
    previous_emails
    # Second, getting ranked responses as per context ------------------
        
        # Building the Langchain vectorstore using chroma collections
    user_vector_store = Chroma(
        client=vector_db_client, 
        collection_name='user'+str(replier_id),
        embedding_function=OpenAIEmbeddings())
        # Getting ranked responses using MMR
    found_rel_emails = await user_vector_store.amax_marginal_relevance_search(sender_email, k=10, fetch_k=10)
    list_rel_emails=[]
    for i, doc in enumerate(found_rel_emails):
        list_rel_emails.append(doc.page_content)
    list_rel_emails
        
    # Setting up LangChain
    llm=ChatOpenAI(temperature=0.1, model=llm_model, openai_api_key=openai_api_key)
    
    template_string="""You are an employee of Enron and receiver of this email: {sender_email},
    Reply to the email as the receiver,
    deriving the context from these relevant emails : {relevant_emails}
    and copying the writing style of the receiver's past emails : {prev_emails}"""
    
    prompt_template = ChatPromptTemplate.from_template(template=template_string)    
    llm_chain=LLMChain(llm=llm, prompt=prompt_template)
    return (llm_chain.run(sender_email=sender_email, prev_emails=previous_emails, relevant_emails=list_rel_emails))

# For no budget cap 

In [56]:
# async def process_all_emails(df):
#     responses = []
#     for index, (idx, row) in enumerate(df.iterrows()):  # enumerate to get index
#         response = await get_email_response_personalized(row['sender'], row['reply_sender'], row['message'], df, openai_api_key, chroma_client)
#         responses.append(response)
#         if (index + 1) % 2 == 0:  # Add sleep after every 2 calls
#             await asyncio.sleep(60)  # Using async sleep
#     return responses
## Run the asynchronous function and get all the responses
# all_responses = await process_all_emails(df_messages_deduped.head(50))

# Storing in parquet till error

In [123]:
async def process_all_emails(df, start_index=0, parquet_file='responses.parquet'):
    responses = []
    
    # Load existing data from the Parquet file if it exists
    if os.path.exists(parquet_file):
        responses_df = pd.read_parquet(parquet_file)
        responses = responses_df.to_dict('records')
    
    for idx, row in df.iloc[start_index:].iterrows():  # Using iloc to skip rows
        try:
            response = await get_email_response_personalized(row['sender'], row['reply_sender'], row['message'], df, openai_api_key, chroma_client)
            responses.append({'index': idx, 'response': response})  # save DataFrame index and response
            
            if (idx + 1) % 2 == 0:  # Add sleep after every 2 calls
                await asyncio.sleep(60)
                
            # Save interim results to the Parquet file
            pd.DataFrame(responses).to_parquet(parquet_file, index=False)
            
        except YourApiLimitError:  # replace YourApiLimitError with the actual exception
            print(f"API limit reached at index {idx}. Stopping execution.")
            break
            
    return responses

# Execute the function
all_responses = await process_all_emails(df_messages_deduped.head(50), start_index=0)


In [124]:
for item in all_responses:
    idx = item['index']
    response = item['response']
    df_messages_deduped.loc[idx, 'responses'] = response

/var/tmp/ipykernel_1081167/2334431273.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messages_deduped.loc[idx, 'responses'] = response


In [125]:
all_responses

[{'index': 0,
  'response': "Hi there,\n\nThanks for reaching out. In response to your email, I'm doing well, thank you for asking. As for Julie, she hasn't left yet and it seems like there was a bit of a mix-up with her school schedule. It's definitely a funny situation, but I'm sure she'll be able to laugh it off eventually.\n\nIn terms of scheduling, you mentioned Friday - are you referring to a specific meeting or event? Let me know and I'll do my best to accommodate.\n\nBest regards,\n\n[Your Name]"},
 {'index': 4,
  'response': 'Dear Sender,\n\nThank you for your email. My family and I are doing well, thank you for asking. As for Julie, I am not sure who you are referring to as there are multiple people with that name in the company. \n\nIn regards to your previous emails, I apologize for not responding sooner. I have been quite busy with work. As for lunch plans, I am available next week on any day except for Monday or Thursday. \n\nThank you for reaching out.\n\nBest regards,\n

In [126]:
# df_messages_deduped.loc[df_messages_deduped.index[:100], 'responses'] = all_responses

In [129]:
# Append the responses to the original dataframe
# df_messages_deduped['responses'] = all_responses

In [130]:
df_messages_deduped

,Unnamed: 0,Email_ID,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message,Sender_Receiver_Emails,Sender_Receiver_Emails_list,sender_replier_thread,responses
0,0,0,1999-06-09 04:18:00-07:00,RE: test,2,5552,[40034],"How about this Friday ? Julie has not left yet. She was 2 days away from leaving to start school on June 7th when she, quite by accident, looked at the school schedule for the first time in several weeks. Now mind you she had lined up a babysitter ,lined up a place to stay, arranged to get a key to start the 7th. The schedule says the school starts the 14th. Pretty funny. How many years will it take her to live this down?. Enron Capital & Trade Resources Corp.",1999-06-09 08:06:00-07:00,40034,[5552],when? how are you and your family? is julie gone?,NaN,[],5552-40034,"Hi there,\n\nThanks for reaching out. In regards to your question about Julie, I'm not sure if she's left yet. Could you clarify? \n\nAlso, I hope you and your family are doing well. When were you thinking of getting together? \n\nBest,\n[Your Name]"
4,4,4,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-11-23 03:58:00-08:00,5552,[40034],Tues.is good. I'll call you.,"['Today is bad. Tommorrow I will call you.', 'Do you have lunch plans today?', ""Really? I'd feel like a mooch. Lets have lunch next week. Any day but Monday or Thurs.""]","[Today is bad. Tommorrow I will call you., Do you have lunch plans today?, Really? I'd feel like a mooch. Lets have lunch next week. Any day but Monday or Thurs.]",40034-5552,"Dear Sender,\n\nThank you for your email. My family and I are doing well, thank you for asking. As for Julie, I am not sure who you are referring to as there are multiple people with that name in the company. \n\nIn regards to the context of your email, I am not sure what you are trying to convey as it seems disconnected from our previous conversations. If there is something specific you need to discuss, please let me know and we can schedule a call or meeting.\n\nBest regards,\n[Your Name]"
10,10,13,2001-11-08 14:18:15-08:00,test,2,39298,[402],test Dave,2001-11-09 09:03:31-08:00,402,[39298],"Dave, Take a look at this before you pass it on. I haven't looked at it in a while. Let me know if you get it and how it looks. Thanks, Kevin",NaN,[],39298-402,"Hi Dave,\n\nI hope this email finds you well. I received your message and wanted to let you know that I have looked at the document you sent me. It seems to be in good shape, but I haven't reviewed it thoroughly yet. I will let you know if I have any further comments or suggestions.\n\nThanks,\nKevin"
11,11,14,2001-11-09 09:03:31-08:00,RE: test,2,402,[39298],"Dave, Take a look at this before you pass it on. I haven't looked at it in a while. Let me know if you get it and how it looks. Thanks, Kevin",2001-11-09 12:06:40-08:00,39298,[402],looks good. Metz says it looks fine too. You'll probably get more questions about Pub Ventures than trading. A lady named Susan Mills should be calling you either tonight or this weekend at home. She'll set up your interview. Everything looks good from this end. Dave,NaN,[],402-39298,"Hi Kevin,\n\nThanks for sending this over. I received it and took a look. Everything seems to be in order, but I'll let you know if I have any further questions or concerns.\n\nBest,\nDave"
12,12,15,2001-11-13 09:08:48-08:00,test,2,39322,[402],test Do You Yahoo!? Find the one for you at Yahoo! Personals http://personals.yahoo.com,2001-11-15 09:27:59-08:00,402,[39322],"sorry, but i didn't get your message until late. i'll call you at home tonight.",NaN,[],39322-402,"Hi Kevin,\n\nThanks for the email. Unfortunately, I'm not sure what you're referring to with the subject line ""test."" In any case, I hope everything is going well with you. \n\nBest,\n[Your Name]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24635,24635,27458,2002-01-31 09:26:01-08:00,Canadian Trader List

In [131]:
# !pip install rouge-score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


In [132]:
def calculate_bleu_score(reference, candidate):
    smoothing_function = SmoothingFunction().method1
    return sentence_bleu([reference], candidate, smoothing_function=smoothing_function)


In [144]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

def calculate_scores(actual, generated):
    # Check if either actual or generated is None, and return None if either is None
    if actual is None or generated is None:
        return {'rouge1': None, 'rougeL': None, 'bleu': None}
    
    # Calculating ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(actual, generated)
    
    # Calculating BLEU score
    bleu_score = sentence_bleu([actual.split()], generated.split())
    
    return {'rouge1': rouge_scores['rouge1'], 'rougeL': rouge_scores['rougeL'], 'bleu': bleu_score}

# Applying the scoring function to rows with non-null responses
df_messages_deduped['evaluation_scores'] = df_messages_deduped.apply(
    lambda row: calculate_scores(str(row['reply_message']), str(row['responses'])) 
    if pd.notnull(row['responses']) and pd.notnull(row['reply_message']) else {'rouge1': None, 'rougeL': None, 'bleu': None}, axis=1
)

# Split the evaluation scores into separate columns
df_messages_deduped[['rouge1_score', 'rougeL_score', 'bleu_score']] = pd.DataFrame(df_messages_deduped['evaluation_scores'].tolist(), index=df_messages_deduped.index)


/var/tmp/ipykernel_1081167/3402930014.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messages_deduped['evaluation_scores'] = df_messages_deduped.apply(
/var/tmp/ipykernel_1081167/3402930014.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messages_deduped[['rouge1_score', 'rougeL_score', 'bleu_score']] = pd.DataFrame(df_messages_deduped['evaluation_scores'].tolist(), index=df_messages_deduped.index)
/var/tmp/ipykernel_1081167/3402930014.py:25: SettingWithCopyWarning: 
A value is trying to be

In [134]:
# Applying the scoring function to rows with non-null responses
df_messages_deduped['evaluation_scores'] = df_messages_deduped.apply(
    lambda row: calculate_scores(str(row['reply_message']), str(row['responses'])) 
    if pd.notnull(row['responses']) and pd.notnull(row['reply_message']) else None, axis=1
)


/var/tmp/ipykernel_1081167/1080931012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messages_deduped['evaluation_scores'] = df_messages_deduped.apply(


In [145]:
df_messages_deduped.head(10)

,Unnamed: 0,Email_ID,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message,Sender_Receiver_Emails,Sender_Receiver_Emails_list,sender_replier_thread,responses,evaluation_scores,rouge1_score,rougeL_score,bleu_score
0,0,0,1999-06-09 04:18:00-07:00,RE: test,2,5552,[40034],"How about this Friday ? Julie has not left yet. She was 2 days away from leaving to start school on June 7th when she, quite by accident, looked at the school schedule for the first time in several weeks. Now mind you she had lined up a babysitter ,lined up a place to stay, arranged to get a key to start the 7th. The schedule says the school starts the 14th. Pretty funny. How many years will it take her to live this down?. Enron Capital & Trade Resources Corp.",1999-06-09 08:06:00-07:00,40034,[5552],when? how are you and your family? is julie gone?,NaN,[],5552-40034,"Hi there,\n\nThanks for reaching out. In regards to your question about Julie, I'm not sure if she's left yet. Could you clarify? \n\nAlso, I hope you and your family are doing well. When were you thinking of getting together? \n\nBest,\n[Your Name]","{'rouge1': (0.15555555555555556, 0.7, 0.2545454545454545), 'rougeL': (0.08888888888888889, 0.4, 0.14545454545454545), 'bleu': 1.245183060581128e-78}","(0.15555555555555556, 0.7, 0.2545454545454545)","(0.08888888888888889, 0.4, 0.14545454545454545)",1.245183e-78
4,4,4,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-11-23 03:58:00-08:00,5552,[40034],Tues.is good. I'll call you.,"['Today is bad. Tommorrow I will call you.', 'Do you have lunch plans today?', ""Really? I'd feel like a mooch. Lets have lunch next week. Any day but Monday or Thurs.""]","[Today is bad. Tommorrow I will call you., Do you have lunch plans today?, Really? I'd feel like a mooch. Lets have lunch next week. Any day but Monday or Thurs.]",40034-5552,"Dear Sender,\n\nThank you for your email. My family and I are doing well, thank you for asking. As for Julie, I am not sure who you are referring to as there are multiple people with that name in the company. \n\nIn regards to the context of your email, I am not sure what you are trying to convey as it seems disconnected from our previous conversations. If there is something specific you need to discuss, please let me know and we can schedule a call or meeting.\n\nBest regards,\n[Your Name]","{'rouge1': (0.043478260869565216, 0.5714285714285714, 0.08080808080808081), 'rougeL': (0.021739130434782608, 0.2857142857142857, 0.04040404040404041), 'bleu': 5.882492291217442e-232}","(0.043478260869565216, 0.5714285714285714, 0.08080808080808081)","(0.021739130434782608, 0.2857142857142857, 0.04040404040404041)",5.882492e-232
10,10,13,2001-11-08 14:18:15-08:00,test,2,39298,[402],test Dave,2001-11-09 09:03:31-08:00,402,[39298],"Dave, Take a look at this before you pass it on. I haven't looked at it in a while. Let me know if you get it and how it looks. Thanks, Kevin",NaN,[],39298-402,"Hi Dave,\n\nI hope this email finds you well. I received your message and wanted to let you know that I have looked at the document you sent me. It seems to be in good shape, but I haven't reviewed it thoroughly yet. I will let you know if I have any further comments or suggestions.\n\nThanks,\nKevin","{'rouge1': (0.3220338983050847, 0.5757575757575758, 0.41304347826086957), 'rougeL': (0.23728813559322035, 0.42424242424242425, 0.30434782608695654), 'bleu': 5.475002426535681e-155}","(0.3220338983050847, 0.5757575757575758, 0.41304347826086957)","(0.23728813559322035, 0.42424242424242425, 0.30434782608695654)",5.475002e-155
11,11,14,2001-11-09 09:03:31-08:00,RE: test,2,402,[39298],"Dave, Take a look at this before you pass it on. I haven't looked at it in a while. Let me know if you get it and how it looks. Thanks, Kevin",2001-11-09 12:06:40-08:00,39298,[402],looks good. Metz says it looks fine too. You'll probably get more questions about Pub Ventures than trading. A 

In [136]:
df_messages_deduped.to_csv('gs://user-scripts-msca310019-capstone-49b3/data/20231025_replies_scores.csv')

In [83]:
# df_messages_deduped['responses'] = all_responses
# df_messages_deduped['evaluation_scores'] = df_messages_deduped.apply(lambda row: calculate_scores(row['reply_message'], row['responses']), axis=1)


In [84]:


# def calculate_rouge_scores(actual, generated):
#     scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
#     scores = scorer.score(actual, generated)
#     return scores

# df_messages_deduped['responses'] = all_responses
# df_messages_deduped['rouge_scores'] = df_messages_deduped.apply(lambda row: calculate_rouge_scores(row['reply_message'], row['responses']), axis=1)



In [149]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

def calculate_scores(actual, generated):
    # Check if either actual or generated is None, and return None if either is None
    if actual is None or generated is None:
        return {'rouge1_precision': None, 'rouge1_recall': None, 'rouge1_f1': None,
                'rougeL_precision': None, 'rougeL_recall': None, 'rougeL_f1': None,
                'bleu_score': None}
    
    # Calculating ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(actual, generated)
    
    # Calculating BLEU score
    bleu_score = sentence_bleu([actual.split()], generated.split())
    
    # Return separate precision, recall, and F1 for ROUGE-1 and ROUGE-L
    return {
        'rouge1_precision': rouge_scores['rouge1'].precision,
        'rouge1_recall': rouge_scores['rouge1'].recall,
        'rouge1_f1': rouge_scores['rouge1'].fmeasure,
        'rougeL_precision': rouge_scores['rougeL'].precision,
        'rougeL_recall': rouge_scores['rougeL'].recall,
        'rougeL_f1': rouge_scores['rougeL'].fmeasure,
        'bleu_score': bleu_score
    }

# Assuming you have a DataFrame df_messages_deduped with 'reply_message' and 'responses' columns

# Applying the scoring function to rows with non-null responses
df_messages_deduped['evaluation_scores'] = df_messages_deduped.apply(
    lambda row: calculate_scores(str(row['reply_message']), str(row['responses'])) 
    if pd.notnull(row['responses']) and pd.notnull(row['reply_message']) else {
        'rouge1_precision': None, 'rouge1_recall': None, 'rouge1_f1': None,
        'rougeL_precision': None, 'rougeL_recall': None, 'rougeL_f1': None,
        'bleu_score': None
    }, axis=1
)

# Split the evaluation scores into separate columns
df_messages_deduped[['rouge1_precision', 'rouge1_recall', 'rouge1_f1',
                     'rougeL_precision', 'rougeL_recall', 'rougeL_f1',
                     'bleu_score']] = pd.DataFrame(df_messages_deduped['evaluation_scores'].tolist(), index=df_messages_deduped.index)

# Calculate the average F1 score for ROUGE-L for the entire dataset
average_rouge1_f1_score = df_messages_deduped['rouge1_f1'].mean()

# Print the result
print(f"Average ROUGE-1 F1-score: {average_rouge1_f1_score:.2f}")



Average ROUGE-1 F1-score: 0.20


/var/tmp/ipykernel_1081167/2299518186.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messages_deduped['evaluation_scores'] = df_messages_deduped.apply(
/var/tmp/ipykernel_1081167/2299518186.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messages_deduped[['rouge1_precision', 'rouge1_recall', 'rouge1_f1',


In [147]:
# Print the result
print(f"Average ROUGE-L F1-score: {average_rougeL_f1_score:.2f}")

Average ROUGE-L F1-score: 0.14
